In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
nba_df = pd.read_csv('nba_games.csv', index_col=0)

In [3]:
# Extract the numerical columns for normalization
numerical_columns = nba_df.columns.difference(['team', 'team_opp', 'season', 'date', 'total', 'won'])

# Normalize the numerical values using Min-Max scaling
scaler = MinMaxScaler()
nba_df[numerical_columns] = scaler.fit_transform(nba_df[numerical_columns])

In [4]:
# Define your custom neural network class
class NBAPredictor(nn.Module):
    def __init__(self, input_size):
        super(NBAPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
X = nba_df.drop(['team', 'team_opp', 'season', 'date', 'total', 'won'], axis=1).values
Y = nba_df['total'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=321)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
Y_train_tensor = torch.FloatTensor(Y_train)
X_test_tensor = torch.FloatTensor(X_test)
Y_test_tensor = torch.FloatTensor(Y_test)

# Create an instance of the neural network
model = NBAPredictor(input_size=X_train.shape[1])

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [6]:
# Training loop
num_epochs = 60
batch_size = 10

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = Y_train_tensor[i:i+batch_size].view(-1, 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    test_pred = model(X_test_tensor)
    mae = mean_absolute_error(Y_test_tensor.numpy(), test_pred.numpy())

print("Mean Absolute Error:", mae)

Mean Absolute Error: 1.6299976


In [7]:
torch.save(model.state_dict(), 'NBA_model.pth')